In [1]:

import pandas as pd
import numpy as np
import os
import skimage
import matplotlib.pyplot as plt

from pathlib import Path

import matplotlib.image as mpimg
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import torchvision.models 
import torch
import torch
import pandas as pd
import numpy as np
import warnings

import os
import re
import shutil
import string

from nltk.corpus import stopwords
from collections import Counter
import nltk
import pandas as pd
import numpy as np

import sklearn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
warnings.simplefilter("ignore")
nltk.download('stopwords')
!python -V

Python 3.10.11


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive

Mounted at /content/gdrive/
/content/gdrive/MyDrive


In [3]:
device = torch.cuda.is_available()
print(f"Is the GPU available? {device}")

Is the GPU available? True


In [4]:
sentim=pd.read_csv("clean_sentim.csv")
sentim

,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...
39104,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39105,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39106,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39107,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


In [5]:
col=["split","tokens","sentiment","word_sentiment"]
grab=sentim[col]
grab

,split,tokens,sentiment,word_sentiment
0,train,"['a', 'plate', 'of', 'delicious', 'food', 'inc...",1,"[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]"
1,train,"['french', 'fries', 'are', 'not', 'a', 'health...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,..."
2,train,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,..."
3,train,"['it', 'was', 'disgusting', 'food', 'not', 'ju...",0,"[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]"
4,train,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...",0,"[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...
39104,test,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]"
39105,test,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]"
39106,test,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
39107,test,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]"


In [6]:
for index, row in grab.iterrows():
    tokens = eval(row['tokens'])
    append_tokens = [0, 0] + tokens + [0, 0]
    grab.at[index, 'tokens'] =append_tokens

In [7]:
grab

,split,tokens,sentiment,word_sentiment
0,train,"[0, 0, a, plate, of, delicious, food, includin...",1,"[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]"
1,train,"[0, 0, french, fries, are, not, a, healthy, fo...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,..."
2,train,"[0, 0, the, plate, has, one, of, my, favorite,...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,..."
3,train,"[0, 0, it, was, disgusting, food, not, just, b...",0,"[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]"
4,train,"[0, 0, a, plate, of, disgusting, food, found, ...",0,"[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...
39104,test,"[0, 0, a, dirty, bathroom, that, has, a, dirty...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]"
39105,test,"[0, 0, a, dirty, bathroom, that, has, a, windo...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]"
39106,test,"[0, 0, a, towel, that, is, on, a, rack, in, a,...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
39107,test,"[0, 0, a, dirty, bathroom, that, has, a, dirty...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]"


In [8]:
grab['words_token_len'] = grab['tokens'].apply(lambda x:len(str(x).split())) 
grab

,split,tokens,sentiment,word_sentiment,words_token_len
0,train,"[0, 0, a, plate, of, delicious, food, includin...",1,"[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",12
1,train,"[0, 0, french, fries, are, not, a, healthy, fo...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",19
2,train,"[0, 0, the, plate, has, one, of, my, favorite,...",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",16
3,train,"[0, 0, it, was, disgusting, food, not, just, b...",0,"[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",12
4,train,"[0, 0, a, plate, of, disgusting, food, found, ...",0,"[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",13
...,...,...,...,...,...
39104,test,"[0, 0, a, dirty, bathroom, that, has, a, dirty...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",16
39105,test,"[0, 0, a, dirty, bathroom, that, has, a, windo...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",13
39106,test,"[0, 0, a, towel, that, is, on, a, rack, in, a,...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",15
39107,test,"[0, 0, a, dirty, bathroom, that, has, a, dirty...",0,"[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",16


In [9]:
new_rows = []
for  j,row in grab.iterrows():
    tokens = row['tokens']
    sentiments =eval(row['word_sentiment'])
    #print(sentiments[0])
    label = row['split']
    for i in range(len(tokens) - 4):
        token_tuple = tuple(tokens[i:i+5])
        sentiment = sentiments[i]
        new_rows.append((token_tuple, sentiment, label))

word_level = pd.DataFrame(new_rows, columns=['tuples', 'word_sentiment', 'split'])
word_level

,tuples,word_sentiment,split
0,"(0, 0, a, plate, of)",0.0,train
1,"(0, a, plate, of, delicious)",0.0,train
2,"(a, plate, of, delicious, food)",0.0,train
3,"(plate, of, delicious, food, including)",1.0,train
4,"(of, delicious, food, including, french)",1.0,train
...,...,...,...
470537,"(bathroom, that, has, a, window)",0.0,test
470538,"(that, has, a, window, in)",0.0,test
470539,"(has, a, window, in, it)",1.0,test
470540,"(a, window, in, it, 0)",0.0,test


Features

In [10]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import normalize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model = model.to(device)

def extract_bert_embeddings(sentence):
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids = torch.tensor(token_ids, dtype=torch.long).unsqueeze(0).to(device)  # Convert to torch.long and move to device
    
    with torch.no_grad():
        outputs = model(token_ids)
        embeddings = outputs.last_hidden_state.squeeze(0)
    
    # Normalize the embeddings
    normalized_embeddings = normalize(embeddings.tolist())
    
    return normalized_embeddings

word_level['tuples'] = word_level['tuples'].astype(str)
word_level['embeds'] = word_level['tuples'].apply(extract_bert_embeddings)



In [14]:
word_level['embeds'] = word_level['embeds'].apply(lambda x: re.sub(r'\s+', ',', x))
word_level=word_level.iloc[:,1:]
word_level

,tuples,word_sentiment,split,embeds
0,"(0, 0, 'a', 'plate', 'of')",0.0,train,"[-0.704883,0.2060582,-0.42128304,...,-0.337813..."
1,"(0, 'a', 'plate', 'of', 'delicious')",0.0,train,"[-0.35149288,0.01229589,-1.2282572,...,-0.5715..."
2,"('a', 'plate', 'of', 'delicious', 'food')",0.0,train,"[-0.8898519,0.2340101,-0.23543039,...,-0.36429..."
3,"('plate', 'of', 'delicious', 'food', 'including')",1.0,train,"[-0.41573092,0.2778056,0.28589982,...,0.081488..."
4,"('of', 'delicious', 'food', 'including', 'fren...",1.0,train,"[-0.70624703,0.77572775,0.17218159,...,0.19065..."
...,...,...,...,...
470537,"('bathroom', 'that', 'has', 'a', 'window')",0.0,test,"[-0.09537123,0.03138053,0.25119308,...,-0.2326..."
470538,"('that', 'has', 'a', 'window', 'in')",0.0,test,"[-0.41970253,-0.20281622,-0.43624753,...,-0.32..."
470539,"('has', 'a', 'window', 'in', 'it')",1.0,test,"[-0.40502307,0.20519122,-0.60786754,...,0.0606..."
470540,"('a', 'window', 'in', 'it', 0)",0.0,test,"[-0.3905728,0.29581422,-0.19451408,...,0.15716..."


In [36]:
word_level['embeds'] = word_level['embeds'].str.replace(r'^\s*,', '', regex=True)
word_level

,tuples,word_sentiment,split,embeds
0,"(0, 0, 'a', 'plate', 'of')",0.0,train,"[-0.704883,0.2060582,-0.42128304,...,-0.337813..."
1,"(0, 'a', 'plate', 'of', 'delicious')",0.0,train,"[-0.35149288,0.01229589,-1.2282572,...,-0.5715..."
2,"('a', 'plate', 'of', 'delicious', 'food')",0.0,train,"[-0.8898519,0.2340101,-0.23543039,...,-0.36429..."
3,"('plate', 'of', 'delicious', 'food', 'including')",1.0,train,"[-0.41573092,0.2778056,0.28589982,...,0.081488..."
4,"('of', 'delicious', 'food', 'including', 'fren...",1.0,train,"[-0.70624703,0.77572775,0.17218159,...,0.19065..."
...,...,...,...,...
470537,"('bathroom', 'that', 'has', 'a', 'window')",0.0,test,"[-0.09537123,0.03138053,0.25119308,...,-0.2326..."
470538,"('that', 'has', 'a', 'window', 'in')",0.0,test,"[-0.41970253,-0.20281622,-0.43624753,...,-0.32..."
470539,"('has', 'a', 'window', 'in', 'it')",1.0,test,"[-0.40502307,0.20519122,-0.60786754,...,0.0606..."
470540,"('a', 'window', 'in', 'it', 0)",0.0,test,"[-0.3905728,0.29581422,-0.19451408,...,0.15716..."


Save file

In [96]:
word_level=word_level.to_csv("word_level_new.csv",index=False)

Read

In [97]:
word_level=pd.read_csv("word_level_new.csv")

In [98]:
word_level

,tuples,word_sentiment,split,embeds
0,"(0, 0, 'a', 'plate', 'of')",0.0,train,"[-0.704883,0.2060582,-0.42128304,...,-0.337813..."
1,"(0, 'a', 'plate', 'of', 'delicious')",0.0,train,"[-0.35149288,0.01229589,-1.2282572,...,-0.5715..."
2,"('a', 'plate', 'of', 'delicious', 'food')",0.0,train,"[-0.8898519,0.2340101,-0.23543039,...,-0.36429..."
3,"('plate', 'of', 'delicious', 'food', 'including')",1.0,train,"[-0.41573092,0.2778056,0.28589982,...,0.081488..."
4,"('of', 'delicious', 'food', 'including', 'fren...",1.0,train,"[-0.70624703,0.77572775,0.17218159,...,0.19065..."
...,...,...,...,...
470537,"('bathroom', 'that', 'has', 'a', 'window')",0.0,test,"[-0.09537123,0.03138053,0.25119308,...,-0.2326..."
470538,"('that', 'has', 'a', 'window', 'in')",0.0,test,"[-0.41970253,-0.20281622,-0.43624753,...,-0.32..."
470539,"('has', 'a', 'window', 'in', 'it')",1.0,test,"[-0.40502307,0.20519122,-0.60786754,...,0.0606..."
470540,"('a', 'window', 'in', 'it', 0)",0.0,test,"[-0.3905728,0.29581422,-0.19451408,...,0.15716..."


Save train,test,valid

In [99]:
train_word=word_level[word_level["split"]=="train"]
test_word=word_level[word_level["split"]=="test"]
valid_word=word_level[word_level["split"]=="val"]

Save files

In [100]:
train_word=train_word.to_csv("train_word.csv",index=False)
test_word=test_word.to_csv("test_word.csv",index=False)
valid_word=valid_word.to_csv("valid_word.csv",index=False)

Load files

In [101]:
train_word=pd.read_csv("train_word.csv")
test_word=pd.read_csv("test_word.csv")
valid_word=pd.read_csv("valid_word.csv")

In [102]:
print(len(train_word))
print(len(test_word))
print(len(valid_word))


228998
195525
46019


In [118]:
from torch.nn.utils.rnn import pad_sequence
class MyDataset(Dataset):
    def __init__(self, trains):
        self.labels = trains['word_sentiment'].tolist()
        self.embeddings = []
        for embedding in trains['embeds']:
            evaluated_embedding = eval(embedding)
            self.embeddings.append(evaluated_embedding)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        label = self.labels[index]
        embedding = self.embeddings[index]

  
        embedding = torch.tensor(embedding)

        return embedding, label


batch_size = 100
train = MyDataset(train_word)

train_dataloader= DataLoader(train, batch_size=batch_size, shuffle=True)
test = MyDataset(test_word)

test_dataloader= DataLoader(test, batch_size=batch_size, shuffle=True)
valid= MyDataset(valid_word)

valid_dataloader= DataLoader(valid, batch_size=batch_size, shuffle=True)

learn_rate = 5
size_of_batch = 64
epoc = 10
non = None

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)
current_accuracy = non

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)
import time

Device cpu


Architecture

In [117]:
import torch
from torch import nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):
    def __init__(self, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()

        self.text_l1 = nn.Linear(embed_dim, 256)
        self.text_l2 = nn.Linear(256, 128)
        self.text_l3 = nn.Linear(128, 64)
        self.text_l4 = nn.Linear(64, 32)

        self.output_combine = nn.Linear(32, num_class)

        self.init_weights()

    def init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                nn.init.zeros_(module.bias)

    def forward(self, text):
        text = F.relu(self.text_l1(text))
        text = F.relu(self.text_l2(text))
        text = F.relu(self.text_l3(text))
        text = F.relu(self.text_l4(text))

        output = self.output_combine(text)

        return output

embeddings, labels = [], []
for batch in train_dataloader:
    embeddings_batch, labels_batch = batch
    embeddings.append(embeddings_batch)
    labels.append(labels_batch)

embed_dim = len(embeddings[0][0])
num_class = len(set(labels))

model = TextClassificationModel(embed_dim, num_class).to(device)
embeddings_tensor = torch.tensor(embeddings).to(device)
labels_tensor = torch.tensor(labels).to(device)


output = model(embeddings_tensor)


In [115]:
def training(dataloader, model, criterion, optimizer):
    model.train()
    current_acc = 0.0
    total_count = 0.0
    log_interval = 100
    start_time = time.time()
    epoch_loss = 0.0
    loss_count = 0
    end_of_epoch_loss = 0
    
    for idx, (labels, embeddings) in enumerate(dataloader):
        labels = torch.tensor(labels).to(device)
        embeddings = torch.tensor(embeddings).to(device)

        optimizer.zero_grad()
        predicted_labels = model(embeddings)
        loss = criterion(predicted_labels, labels)
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 0.1)
        optimizer.step()

        max_predicted_each_class = predicted_labels.argmax(1)
        compare = max_predicted_each_class == labels
        correct_pred = compare.sum().item()
        current_acc += correct_pred
        total_count += labels.size(0)

        loss_count += loss.item() * labels.size(0)
        epoch_loss += loss.item() * labels.size(0)

        if idx % log_interval == 0:
            if idx > 0:
                elapsed = time.time() - start_time
                current_loss = loss_count / total_count  
                print('< epoch {:3d} < {:5d}/{:5d} batches '
                      '< accuracy {:8.3f} <loss {:.2f}'.format(ep, idx, len(dataloader),
                                                                 current_acc / total_count, current_loss))
                current_acc = 0
                total_count = 0
                loss_count = 0
                start_time = time.time()

    epoch_loss /= len(dataloader.dataset)
    end_of_epoch_loss = epoch_loss
    print("end of epoch loss Train {:.2f}".format(end_of_epoch_loss))

def evaluate(dataloader, model, criterion):
    model.eval()
    predicted_labels = []
    true_labels = []
    current_accu = 0.0
    total_count = 0.0
    acc_prop = 0
    loss_count=0
    
    with torch.no_grad():
        for id, (labels, embeddings) in enumerate(dataloader):
            labels = torch.tensor(labels).to(device)
            embeddings = torch.tensor(embeddings).to(device)

            predicted_label = model(embeddings)
            loss = criterion(predicted_label, labels)
            max_predicted_each_class = predicted_label.argmax(1)
            compare = max_predicted_each_class == labels
            correct_pred = compare.sum().item()
            current_accu += correct_pred
            total_count += labels.size(0)
            loss_count += loss.item() * labels.size(0)

            predicted_labels.extend(max_predicted_each_class.tolist())
            true_labels.extend(labels.tolist())

    valid_loss=loss_count / len(dataloader.dataset)
    acc_prop = current_accu / total_count
    #print('epoch validation loss {:.3f}'.format(loss_count / len(dataloader.dataset)) )
    return valid_loss, acc_prop, predicted_labels, true_labels


In [116]:
from sklearn.metrics import f1_score,accuracy_score

In [ ]:



# Training loop
#end_of_epoch_loss = 0
predicted_labels_all = []
true_labels_all = []

for ep in range(1, epoc):
    #end_of_epoch_loss = 0
    starting_time = time.time()
    training(train_dataloader, model, criterion, optimizer)
    validate_loss,validate_accuracy,predicted_labels_epoch, true_labels_epoch = evaluate(valid_dataloader, model, criterion)

   
    if current_accuracy is not None:
        if current_accuracy > validate_accuracy:
            scheduler.step()
    else:
        current_accuracy = validate_accuracy

    print('=' * 70)
    print('< end of epoch {:3d} < time: {:5.2f}s < '
          'valid accuracy {:8.3f} '.format(ep, time.time() - starting_time, validate_accuracy))
    print("<valid loss {:.3f}".format (validate_loss))
    print('=' * 70)
    

    predicted_labels_all.extend(predicted_labels_epoch)
    true_labels_all.extend(true_labels_epoch)








In [119]:
f1 = f1_score( true_labels_all ,predicted_labels_all, average='macro')
print("-------------------- F1 score--------------------",f1)
accuracy = accuracy_score(true_labels_all, predicted_labels_all)
print("-------------------- Accuracy --------------------",accuracy )

-------------------- F1 score-------------------- 0.459
-------------------- Accuracy -------------------- 0.53


Thankyou